In the cell below we will load the core libraries we will be using for this workshop and setting some sensible defaults for our plot size and resolution.

In [1]:

#Data Libraries: 
import numpy as np 
import pandas as pd
import random 
import math 

#Misc Library
import os 

#plotting library
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80







For the purpose of the project - We make functions - triangular, capacity_profit, demand_profit, capacity_difference. 

The triangular function will deal with the first part of the report which is to make 10000 simulations to estimate the base values of the model. Further, the capacity_profit, demand_profit, capacity_difference are used for market stabilty analysis as the results from the paramters are used to make the graphs 

In [ ]:


def triangular(amt, objective_wieght, rho_inf, rho_dap):
    '''
    Using this function we generate 10000 parameters for our simulated more. We neet to input amt, objective_wieght, rho_inf and       rho_dap 

    Our output in this case will be a data frame with 10000 parameters. 
    '''


    obj = [objective_wieght, objective_wieght]

    #k1_cap = [k_inf*amt, k_inf*amt]#only for infranix
    #k2_cap = [k_dap*amt, k_dap*amt] #only for daptacel 

    #P1 = [P_inf*amt, P_dap*amt]
    #P2 = [P_inf*amt, P_dap*amt]
    rho_inf = [rho_inf]
    rho_dap = [rho_dap]
    k1_cap_int = 33260
    k2_cap_int = 33260 
    p1_int = 2.65*amt
    p2_int = 2.65*amt 
    dem_int = amt 
    gam_int = 1 
    obj_int =5 
    number_of_samples = 100000


    #.triangular value low, high, mode; here our mode will be the base values 
    K_inf = np.random.triangular(2.837*amt,4.034*amt, 4.034*amt , number_of_samples)
    K_dap = np.random.triangular(2.837*amt, 4.034*amt, 4.034*amt, number_of_samples)
    P_inf = np.random.triangular(26.5*amt, 39.8 *amt, 53*amt, number_of_samples)
    P_dap = np.random.triangular(26.5*amt, 45.1*amt, 53*amt, number_of_samples)
    D = np.random.triangular(4*amt, 4.034*amt, 7.832 *amt, number_of_samples)                 
    G = np.random.triangular(0.01, 0.25, 0.5, number_of_samples)
    O = np.repeat(obj[0], number_of_samples)
    rho_1 = np.repeat(rho_inf, number_of_samples)  
    rho_2 = np.repeat(rho_dap, number_of_samples)
    a_priv = np.reshape([(1.03 + 2.389/(1+G))*amt],(100000,))
    a_pub = np.reshape([(1.365 + 1.553/(1+G))*amt],(100000,))
    b = [100000/((1+G)*(1-G)),100000/((1+G)*(1-G))]
    c = [G*b[0],G*b[0]]
    p_priv =np.reshape([(34.19 + 10.3*G)*(1-G)/(2-G)],(100000,))
    q_priv =np.reshape([((3.149 + 1.03*G)*10**6)/((1+G)*(2-G))],(100000,))
    h = ((3.149 + 1.03*G)*10**6)/G
    v =  ((1 + G)**(0.5))*(2 -G) 
    k = (1 - 2*(1 - G)**(0.5)/v)
    j = h*k
    U = np.reshape([j],(100000,))

    grid = np.array([K_inf, K_dap, P_inf, P_dap, D, G, O, rho_1, rho_2, a_priv, a_pub, b[0],b[1],c[0],c[1],p_priv,q_priv, U])

    grid

    df = pd.DataFrame(grid)
    df = df.T
    df.columns = ("m1_cap","m2_cap","m1_prof","m2_prof","dem","prod_sim","mu","m1_infl","m2_infl","a_priv","a_pub","b_pub","b_priv", "c_pub", "c_priv","p_priv","q_priv","U")
    
    return df

#These are the default values for triangular distribution!
df = triangular(10**6, 10**-4, 18.62, 18.02)

In [19]:
 
 def capacity_profit(amt,th,OBJ):

    DEM = [4 *amt , 4 *amt]  #total annual US demand
    DEM_INT = amt            #demand interval

    GAM = [0.25,0.25]        #product similarity in [0,1], where 1=identical and 0 is totally dissimilar products. 
    GAM_INT = 1

    OBJ_GOVT = [OBJ, OBJ]    #objective function weight for gov't cost
    OBJ_GOVT_INT = 5         #govt weight interval 

    M1_INFLATION = [18.62]   # Infanrix price upper bound
    M2_INFLATION = [18.02]   #Daptacel price uppr bound 
 

    TOTAL_PROF = 84.9 *amt
    PROF = [32.5*amt, 52.4*amt]
    PROF_INT = 398*th
    TOTAL_CAP = 4.034*2*amt
    CAP = [2.034*amt, 4.034*amt]
    CAP_INT = 40*th

    grid=np.array([(K,TOTAL_CAP-K,P,TOTAL_PROF - P,D,G,OG,I1,I2)
           for K in np.arange(CAP[0],CAP[1]+CAP_INT,CAP_INT)
           for P in np.arange(PROF[0],PROF[1]+PROF_INT,PROF_INT)
           for D in np.arange(DEM[0],DEM[1]+DEM_INT,DEM_INT)
           for G in np.arange(GAM[0],GAM[1]+GAM_INT,GAM_INT)
           for OG in np.arange(OBJ_GOVT[0],OBJ_GOVT[1]+OBJ_GOVT_INT,OBJ_GOVT_INT)
           for I1 in M1_INFLATION
           for I2 in M2_INFLATION
        ])

    df = pd.DataFrame(grid)
    df.columns = ("m1_cap","m2_cap","m1_prof","m2_prof","dem","prod_sim","obj_govt","m1_infl","m2_infl")


    return df

 df = capacity_profit(10**6, 10**3, 10**-4)

In [3]:
 def demand_profit(amt,th,OBJ):

    DEM = [4 *amt , 4 *amt]  #total annual US demand
    DEM_INT = amt            #demand interval

    GAM = [0.25,0.25]        #product similarity in [0,1], where 1=identical and 0 is totally dissimilar products. 
    GAM_INT = 1

    OBJ_GOVT = [OBJ, OBJ]    #objective function weight for gov't cost
    OBJ_GOVT_INT = 5         #govt weight interval 

    M1_INFLATION = [18.62]   # Infanrix price upper bound
    M2_INFLATION = [18.02]   #Daptacel price uppr bound 
 

    TOTAL_PROF = 84.9 *amt
    PROF = [32.5*amt, 52.4*amt]
    PROF_INT = 398*th
    TOTAL_CAP = 4.034*2*amt
    CAP = [2.034*amt, 4.034*amt]
    CAP_INT = 40*th

    M1_CAP = [4.034 *amt, 4.034 *amt] #rec              # production capacity min, max
    M1_CAP_INT = 23480                                 # production capacity interval
    M2_CAP = [4.034 *amt, 4.034 *amt] #eng
    M2_CAP_INT = 23480

    TOTAL_PROF = 84.9 *amt
    PROF = [28.45 *amt, 42.45 *amt]
    PROF_INT = 2*amt

    grid=np.array([(K1,K2,P,TOTAL_PROF - P,D,G,OG,I1,I2)
           for K1 in np.arange(M1_CAP[0],M1_CAP[1]+M1_CAP_INT,M1_CAP_INT)
           for K2 in np.arange(M2_CAP[0],M2_CAP[1]+M2_CAP_INT,M2_CAP_INT)
           for P in np.arange(PROF[0],PROF[1]+PROF_INT,PROF_INT)
           for D in np.arange(DEM[0],DEM[1]+DEM_INT,DEM_INT)
           for G in np.arange(GAM[0],GAM[1]+GAM_INT,GAM_INT)
           for OG in np.arange(OBJ_GOVT[0],OBJ_GOVT[1]+OBJ_GOVT_INT,OBJ_GOVT_INT)
           for I1 in M1_INFLATION
           for I2 in M2_INFLATION
        ])

    df = pd.DataFrame(grid)
    df.columns = ("m1_cap","m2_cap","m1_prof","m2_prof","dem","prod_sim","obj_govt","m1_infl","m2_infl")


    return df

df = demand_profit(10**6, 10**3, 10**-4)

In [8]:
def capacity_difference(amt,th,OBJ):

    DEM = [4 *amt , 4 *amt]  #total annual US demand
    DEM_INT = amt            #demand interval

    GAM = [0.25,0.25]        #product similarity in [0,1], where 1=identical and 0 is totally dissimilar products. 
    GAM_INT = 1

    OBJ_GOVT = [OBJ, OBJ]    #objective function weight for gov't cost
    OBJ_GOVT_INT = 5         #govt weight interval 

    M1_INFLATION = [18.62]   # Infanrix price upper bound
    M2_INFLATION = [18.02]   #Daptacel price uppr bound 
 

    TOTAL_PROF = 84.9 *amt
    PROF = [32.5*amt, 52.4*amt]
    PROF_INT = 398*th
    TOTAL_CAP = 4.034*2*amt
    CAP = [2.034*amt, 4.034*amt]
    CAP_INT = 40*th

    
    TOTAL_CAP = 4.034*2 *amt
    CAP = [2.534 *amt, 4.034 *amt]
    CAP_INT = 250000

    M1_PROF = [42.45 *amt, 42.45 *amt] #rec              # target profit min, max
    M1_PROF_INT = 530 *th                            # target profit interval
    M2_PROF = [42.45 *amt, 42.45 *amt] #eng
    M2_PROF_INT = 530 *th


    grid=np.array([(K,TOTAL_CAP-K,P1,P2,D,G,OG,I1,I2)
           for K in np.arange(CAP[0],CAP[1]+CAP_INT,CAP_INT)
           for P1 in np.arange(M1_PROF[0],M1_PROF[1]+M1_PROF_INT,M1_PROF_INT)
           for P2 in np.arange(M2_PROF[0],M2_PROF[1]+M2_PROF_INT,M2_PROF_INT)
           for D in np.arange(DEM[0],DEM[1]+DEM_INT,DEM_INT)
           for G in np.arange(GAM[0],GAM[1]+GAM_INT,GAM_INT)
           for OG in np.arange(OBJ_GOVT[0],OBJ_GOVT[1]+OBJ_GOVT_INT,OBJ_GOVT_INT)
           for I1 in M1_INFLATION
           for I2 in M2_INFLATION
        ])

    df = pd.DataFrame(grid)
    df.columns = ("m1_cap","m2_cap","m1_prof","m2_prof","dem","prod_sim","obj_govt","m1_infl","m2_infl")


    return df

df = capacity_difference(10**6, 10**3, 10**-4)

In [9]:

gamma = df.prod_sim

a = [1365000+(1553000/(1+gamma)), 1030000+(2389000/(1+gamma))]     # pub, priv (DTAP)
b = [100000/((1+gamma)*(1-gamma)), 100000/((1+gamma)*(1-gamma))]
c = [gamma*b[0], gamma*b[1]]

df['a_pub'] = a[0]; df['a_priv'] = a[1]
df['b_pub'] = b[0]; df['b_priv'] = b[1]
df['c_pub'] = c[0]; df['c_priv'] = c[1]


In [10]:
#Generates commands input for NEOS
file = open('test2.txt', 'w')

for i in range(df.shape[0]):
    file.writelines('solve;\ndisplay gamma;\ndisplay demand;\ndisplay mu;'\
                    '\ndisplay price;\ndisplay quant;\ndisplay z;\ndisplay PubCost;\n\n')
    
    file.writelines('reset data price, quant, z, PubCost;\n\n')
    file.writelines('let gamma := {:.2f};\n'.format(df.prod_sim[i]));
    file.writelines('let demand := {:.0f};\n'.format(df.dem[i]));
    file.writelines("let a['public'] := {:.4f};\n".format(df.a_pub[i]));   
    file.writelines("let a['private'] := {:.4f};\n".format(df.a_priv[i]));
    file.writelines("let b['public'] := {:.4f};\n".format(df.b_pub[i]));   
    file.writelines("let b['private'] := {:.4f};\n".format(df.b_priv[i]));    
    file.writelines("let c['public'] := {:.4f};\n".format(df.c_pub[i]));   
    file.writelines("let c['private'] := {:.4f};\n".format(df.c_priv[i]));
    file.writelines("let K['x'] := {:.0f};\n".format(df.m1_cap[i]));   
    file.writelines("let K['y'] := {:.0f};\n".format(df.m2_cap[i]));
    file.writelines("let P['x'] := {:.0f};\n".format(df.m1_prof[i]));   
    file.writelines("let P['y'] := {:.0f};\n".format(df.m2_prof[i]));
    file.writelines("let rho['x'] := {:.2f};\n".format(df.m1_infl[i]));   
    file.writelines("let rho['y'] := {:.2f};\n\n".format(df.m2_infl[i])); 

file.writelines('solve;\ndisplay gamma;\ndisplay demand;\ndisplay mu;'\
                    '\ndisplay price;\ndisplay quant;\ndisplay z;\ndisplay PubCost;\n\n')
file.close()